In [1]:
import torch
import numpy as np
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import MNIST
from skorch import NeuralNetClassifier
from modAL.models import ActiveLearner
from batchbald_redux import batchbald
from acquisition_functions import *
import os
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import backend as K
import keras
from keras.datasets import mnist

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
LATENT_DIMS = [2, 4, 8, 16, 32, 64, 128, 256, 512]
MAX_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EXPERIMENT_COUNT = 3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 512
ORIGINAL_DIM = 784
results_path = 'results\\mnist_vanillavae_dbal'
epsilon_std = 1.0

if not os.path.exists(results_path):
    os.makedirs(results_path)
'''
ACQ_FUNCS = {
    "var_ratios": var_ratios,
    "mean_std": mean_std,
    "max_entropy": max_entropy,
    "bald": bald,
    "uniform": uniform,
    "batch_bald": batch_bald
}
'''
ACQ_FUNCS = {
    "max_entropy": max_entropy
}

In [4]:
class MLP_REG(nn.Module):
    def __init__(self, latent_dim):
        super(MLP_REG, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(p=0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 10),)
        
        
    def forward(self, x):
        return self.layers(x)


### read training data

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255.
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255.


In [6]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [7]:
def active_learning_procedure(query_strategy,
                              X_test,
                              y_test,
                              X_pool,
                              y_pool,
                              X_initial,
                              y_initial,
                              estimator,
                              n_queries=198,
                              n_instances=10):
    learner = ActiveLearner(estimator=estimator,
                            X_training=X_initial,
                            y_training=y_initial,
                            query_strategy=query_strategy,
                           )
    perf_hist = [learner.score(X_test, y_test)]
    active_pool_size = [len(X_initial)]
    pool_size = len(X_initial)
    for index in range(n_queries):
        query_idx, query_instance = learner.query(X_pool, n_instances)
        learner.teach(X_pool[query_idx], y_pool[query_idx])
        X_pool = np.delete(X_pool, query_idx, axis=0)
        y_pool = np.delete(y_pool, query_idx, axis=0)
        model_accuracy = learner.score(X_test, y_test)
        pool_size = pool_size + n_instances
        print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))
        perf_hist.append(model_accuracy)
        active_pool_size.append(pool_size)
    return perf_hist, active_pool_size

In [8]:
for latent_dim in LATENT_DIMS:

    # encoder architecture
    x = Input(shape=(ORIGINAL_DIM,))
    encoder_h = Dense(HIDDEN_DIM, activation='relu')(x)
    z_mean = Dense(latent_dim)(encoder_h)

    # encoder to generate latent variables from input
    encoder = Model(x, z_mean)

    encoder.load_weights(r"C:\Users\pinar\OneDrive\Masaüstü\masterthesis\src\Generative Models\mnist\mnist_vanillavae\mnist_vanillavae_z_mean_{latent_dim}.h5".format(latent_dim=latent_dim))
    
    X_train_enhanced = encoder.predict(X_train, batch_size=32)
    X_test_enhanced = encoder.predict(X_test, batch_size=32)
    
    for exp_iter in range(EXPERIMENT_COUNT):
        np.random.seed(exp_iter)
        initial_idx = np.array([],dtype=int)
        for i in range(10):
            idx = np.random.choice(np.where(y_train==i)[0], size=2, replace=False)
            initial_idx = np.concatenate((initial_idx, idx))

        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train_enhanced[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train_enhanced, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(latent_dim).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test_enhanced,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join(results_path, "{func_name}_latent_dim_{latent_dim}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter, latent_dim=latent_dim))
            np.save(file_name, (acc_arr, dataset_size_arr))
        '''
        for func_name, acquisition_func in ACQ_FUNCS.items():  
            X_initial = X_train[initial_idx]
            y_initial = y_train[initial_idx]

            X_pool = np.delete(X_train, initial_idx, axis=0)
            y_pool = np.delete(y_train, initial_idx, axis=0)

            model = MLP_REG(ORIGINAL_DIM).to(DEVICE)

            estimator = NeuralNetClassifier(model,
                                          max_epochs=MAX_EPOCHS,
                                          batch_size=BATCH_SIZE,
                                          lr=LEARNING_RATE,
                                          optimizer=torch.optim.Adam,
                                          criterion=torch.nn.CrossEntropyLoss,
                                          train_split=None,
                                          verbose=0,
                                          device=DEVICE)



            acc_arr, dataset_size_arr = active_learning_procedure(acquisition_func,
                                                              X_test,
                                                              y_test,
                                                              X_pool,
                                                              y_pool,
                                                              X_initial,
                                                              y_initial,
                                                              estimator,)
            file_name = os.path.join("results\\mnist_dbal", "{func_name}_exp_{exp_iter}.npy".format(func_name=func_name, exp_iter=exp_iter))
            np.save(file_name, (acc_arr, dataset_size_arr))
            '''

Accuracy after query 1: 0.5172
Accuracy after query 2: 0.5337
Accuracy after query 3: 0.5470
Accuracy after query 4: 0.5116
Accuracy after query 5: 0.5219
Accuracy after query 6: 0.5061
Accuracy after query 7: 0.5230
Accuracy after query 8: 0.5094
Accuracy after query 9: 0.5221
Accuracy after query 10: 0.5324
Accuracy after query 11: 0.5267
Accuracy after query 12: 0.5134
Accuracy after query 13: 0.5117
Accuracy after query 14: 0.5193
Accuracy after query 15: 0.5033
Accuracy after query 16: 0.5239
Accuracy after query 17: 0.5286
Accuracy after query 18: 0.5174
Accuracy after query 19: 0.5328
Accuracy after query 20: 0.5160
Accuracy after query 21: 0.5309
Accuracy after query 22: 0.5226
Accuracy after query 23: 0.5173
Accuracy after query 24: 0.5151
Accuracy after query 25: 0.4998
Accuracy after query 26: 0.4996
Accuracy after query 27: 0.5112
Accuracy after query 28: 0.5018
Accuracy after query 29: 0.5181
Accuracy after query 30: 0.5125
Accuracy after query 31: 0.5148
Accuracy after qu

Accuracy after query 57: 0.4772
Accuracy after query 58: 0.4860
Accuracy after query 59: 0.4898
Accuracy after query 60: 0.5005
Accuracy after query 61: 0.5021
Accuracy after query 62: 0.5381
Accuracy after query 63: 0.5273
Accuracy after query 64: 0.5213
Accuracy after query 65: 0.5329
Accuracy after query 66: 0.5383
Accuracy after query 67: 0.5298
Accuracy after query 68: 0.5324
Accuracy after query 69: 0.5066
Accuracy after query 70: 0.5393
Accuracy after query 71: 0.5314
Accuracy after query 72: 0.5455
Accuracy after query 73: 0.5640
Accuracy after query 74: 0.5441
Accuracy after query 75: 0.5528
Accuracy after query 76: 0.5610
Accuracy after query 77: 0.5660
Accuracy after query 78: 0.5565
Accuracy after query 79: 0.5534
Accuracy after query 80: 0.5679
Accuracy after query 81: 0.5512
Accuracy after query 82: 0.5528
Accuracy after query 83: 0.5740
Accuracy after query 84: 0.5375
Accuracy after query 85: 0.5677
Accuracy after query 86: 0.5630
Accuracy after query 87: 0.5602
Accuracy

Accuracy after query 112: 0.6583
Accuracy after query 113: 0.6500
Accuracy after query 114: 0.6392
Accuracy after query 115: 0.6484
Accuracy after query 116: 0.6372
Accuracy after query 117: 0.6435
Accuracy after query 118: 0.6499
Accuracy after query 119: 0.6390
Accuracy after query 120: 0.6412
Accuracy after query 121: 0.6442
Accuracy after query 122: 0.6471
Accuracy after query 123: 0.6505
Accuracy after query 124: 0.6428
Accuracy after query 125: 0.6464
Accuracy after query 126: 0.6502
Accuracy after query 127: 0.6519
Accuracy after query 128: 0.6474
Accuracy after query 129: 0.6487
Accuracy after query 130: 0.6521
Accuracy after query 131: 0.6370
Accuracy after query 132: 0.6547
Accuracy after query 133: 0.6503
Accuracy after query 134: 0.6445
Accuracy after query 135: 0.6397
Accuracy after query 136: 0.6445
Accuracy after query 137: 0.6431
Accuracy after query 138: 0.6362
Accuracy after query 139: 0.6424
Accuracy after query 140: 0.6490
Accuracy after query 141: 0.6350
Accuracy a

Accuracy after query 166: 0.8586
Accuracy after query 167: 0.8595
Accuracy after query 168: 0.8591
Accuracy after query 169: 0.8571
Accuracy after query 170: 0.8585
Accuracy after query 171: 0.8577
Accuracy after query 172: 0.8556
Accuracy after query 173: 0.8562
Accuracy after query 174: 0.8594
Accuracy after query 175: 0.8595
Accuracy after query 176: 0.8556
Accuracy after query 177: 0.8622
Accuracy after query 178: 0.8611
Accuracy after query 179: 0.8583
Accuracy after query 180: 0.8605
Accuracy after query 181: 0.8604
Accuracy after query 182: 0.8607
Accuracy after query 183: 0.8613
Accuracy after query 184: 0.8586
Accuracy after query 185: 0.8617
Accuracy after query 186: 0.8608
Accuracy after query 187: 0.8589
Accuracy after query 188: 0.8605
Accuracy after query 189: 0.8571
Accuracy after query 190: 0.8633
Accuracy after query 191: 0.8624
Accuracy after query 192: 0.8605
Accuracy after query 193: 0.8611
Accuracy after query 194: 0.8603
Accuracy after query 195: 0.8625
Accuracy a

Accuracy after query 23: 0.8171
Accuracy after query 24: 0.8203
Accuracy after query 25: 0.8365
Accuracy after query 26: 0.8420
Accuracy after query 27: 0.8407
Accuracy after query 28: 0.8313
Accuracy after query 29: 0.8242
Accuracy after query 30: 0.8256
Accuracy after query 31: 0.8247
Accuracy after query 32: 0.8239
Accuracy after query 33: 0.8257
Accuracy after query 34: 0.8261
Accuracy after query 35: 0.8244
Accuracy after query 36: 0.8298
Accuracy after query 37: 0.8330
Accuracy after query 38: 0.8315
Accuracy after query 39: 0.8290
Accuracy after query 40: 0.8383
Accuracy after query 41: 0.8354
Accuracy after query 42: 0.8355
Accuracy after query 43: 0.8397
Accuracy after query 44: 0.8371
Accuracy after query 45: 0.8410
Accuracy after query 46: 0.8341
Accuracy after query 47: 0.8407
Accuracy after query 48: 0.8359
Accuracy after query 49: 0.8357
Accuracy after query 50: 0.8352
Accuracy after query 51: 0.8377
Accuracy after query 52: 0.8335
Accuracy after query 53: 0.8335
Accuracy

...

Accuracy after query 145: 0.9714
Accuracy after query 146: 0.9731
Accuracy after query 147: 0.9701
Accuracy after query 148: 0.9709
Accuracy after query 149: 0.9716
Accuracy after query 150: 0.9717
Accuracy after query 151: 0.9699
Accuracy after query 152: 0.9707
Accuracy after query 153: 0.9725
Accuracy after query 154: 0.9726
Accuracy after query 155: 0.9697
Accuracy after query 156: 0.9727
Accuracy after query 157: 0.9745
Accuracy after query 158: 0.9733
Accuracy after query 159: 0.9708
Accuracy after query 160: 0.9740
Accuracy after query 161: 0.9728
Accuracy after query 162: 0.9716
Accuracy after query 163: 0.9732
Accuracy after query 164: 0.9735
Accuracy after query 165: 0.9739
Accuracy after query 166: 0.9739
Accuracy after query 167: 0.9732
Accuracy after query 168: 0.9729
Accuracy after query 169: 0.9739
Accuracy after query 170: 0.9737
Accuracy after query 171: 0.9734
Accuracy after query 172: 0.9743
Accuracy after query 173: 0.9732
Accuracy after query 174: 0.9713
Accuracy a

Accuracy after query 1: 0.5125
Accuracy after query 2: 0.5568
Accuracy after query 3: 0.5927
Accuracy after query 4: 0.6144
Accuracy after query 5: 0.6725
Accuracy after query 6: 0.6905
Accuracy after query 7: 0.7413
Accuracy after query 8: 0.7622
Accuracy after query 9: 0.8015
Accuracy after query 10: 0.8369
Accuracy after query 11: 0.8302
Accuracy after query 12: 0.8418
Accuracy after query 13: 0.8446
Accuracy after query 14: 0.8496
Accuracy after query 15: 0.8507
Accuracy after query 16: 0.8515
Accuracy after query 17: 0.8524
Accuracy after query 18: 0.8504
Accuracy after query 19: 0.8596
Accuracy after query 20: 0.8675
Accuracy after query 21: 0.8590
Accuracy after query 22: 0.8502
Accuracy after query 23: 0.8719
Accuracy after query 24: 0.8825
Accuracy after query 25: 0.8727
Accuracy after query 26: 0.9004
Accuracy after query 27: 0.8919
Accuracy after query 28: 0.9021
Accuracy after query 29: 0.9002
Accuracy after query 30: 0.8946
Accuracy after query 31: 0.9025
Accuracy after qu

Accuracy after query 57: 0.9262
Accuracy after query 58: 0.9294
Accuracy after query 59: 0.9329
Accuracy after query 60: 0.9285
Accuracy after query 61: 0.9264
Accuracy after query 62: 0.9330
Accuracy after query 63: 0.9388
Accuracy after query 64: 0.9396
Accuracy after query 65: 0.9391
Accuracy after query 66: 0.9418
Accuracy after query 67: 0.9405
Accuracy after query 68: 0.9388
Accuracy after query 69: 0.9431
Accuracy after query 70: 0.9418
Accuracy after query 71: 0.9402
Accuracy after query 72: 0.9425
Accuracy after query 73: 0.9399
Accuracy after query 74: 0.9440
Accuracy after query 75: 0.9468
Accuracy after query 76: 0.9512
Accuracy after query 77: 0.9493
Accuracy after query 78: 0.9483
Accuracy after query 79: 0.9511
Accuracy after query 80: 0.9457
Accuracy after query 81: 0.9484
Accuracy after query 82: 0.9464
Accuracy after query 83: 0.9554
Accuracy after query 84: 0.9510
Accuracy after query 85: 0.9513
Accuracy after query 86: 0.9487
Accuracy after query 87: 0.9505
Accuracy

Accuracy after query 112: 0.9602
Accuracy after query 113: 0.9615
Accuracy after query 114: 0.9587
Accuracy after query 115: 0.9600
Accuracy after query 116: 0.9623
Accuracy after query 117: 0.9621
Accuracy after query 118: 0.9644
Accuracy after query 119: 0.9653
Accuracy after query 120: 0.9641
Accuracy after query 121: 0.9633
Accuracy after query 122: 0.9626
Accuracy after query 123: 0.9629
Accuracy after query 124: 0.9628
Accuracy after query 125: 0.9656
Accuracy after query 126: 0.9636
Accuracy after query 127: 0.9663
Accuracy after query 128: 0.9673
Accuracy after query 129: 0.9635
Accuracy after query 130: 0.9633
Accuracy after query 131: 0.9642
Accuracy after query 132: 0.9624
Accuracy after query 133: 0.9632
Accuracy after query 134: 0.9667
Accuracy after query 135: 0.9655
Accuracy after query 136: 0.9648
Accuracy after query 137: 0.9657
Accuracy after query 138: 0.9645
Accuracy after query 139: 0.9656
Accuracy after query 140: 0.9698
Accuracy after query 141: 0.9666
Accuracy a

Accuracy after query 166: 0.9717
Accuracy after query 167: 0.9697
Accuracy after query 168: 0.9698
Accuracy after query 169: 0.9708
Accuracy after query 170: 0.9695
Accuracy after query 171: 0.9710
Accuracy after query 172: 0.9686
Accuracy after query 173: 0.9702
Accuracy after query 174: 0.9715
Accuracy after query 175: 0.9690
Accuracy after query 176: 0.9727
Accuracy after query 177: 0.9706
Accuracy after query 178: 0.9707
Accuracy after query 179: 0.9701
Accuracy after query 180: 0.9721
Accuracy after query 181: 0.9710
Accuracy after query 182: 0.9674
Accuracy after query 183: 0.9699
Accuracy after query 184: 0.9726
Accuracy after query 185: 0.9713
Accuracy after query 186: 0.9725
Accuracy after query 187: 0.9709
Accuracy after query 188: 0.9730
Accuracy after query 189: 0.9718
Accuracy after query 190: 0.9726
Accuracy after query 191: 0.9739
Accuracy after query 192: 0.9740
Accuracy after query 193: 0.9726
Accuracy after query 194: 0.9736
Accuracy after query 195: 0.9732
Accuracy a